# LanceDB Debugging and Exploration

In [38]:
import sys
from pathlib import Path

# Get the project root directory
project_root = Path.cwd().parent  # Adjust if your notebook is nested differently
sys.path.append(str(project_root))

In [39]:
import lancedb
from lancedb.table import Table
from lancedb.pydantic import pydantic_to_schema

from models.data_models import VideoData, VideoSegmentData
from mm_vector_stores.multimodal_lancedb import MultiModalLanceDB
from mm_emeddings.openclip_embeddings import OpenClipEmbeddings
from utils.logger import logger

In [40]:
db = MultiModalLanceDB(uri="/home/zalasyu/Documents/projects/multimodal_chatbot/data/multimodal_lancedb")
embedder = OpenClipEmbeddings()

In [41]:
video_segments_table = db.get_table("VideoSegments")

In [42]:
video_segments_table.head()

pyarrow.Table
id: int64 not null
parent_video_id: string not null
parent_video_path: string not null
parent_audio_path: string not null
parent_vtt_path: string not null
video_segment_path: string not null
video_segment_transcript_path: string not null
frame_path: string not null
transcript: string not null
enriched_transcript: string not null
duration_ms: double not null
start_ms: double not null
mid_ms: double not null
end_ms: double not null
embeddings: fixed_size_list<item: float>[512] not null
  child 0, item: float
embeddings_from_text: fixed_size_list<item: float>[512] not null
  child 0, item: float
----
id: [[0,1,2,3,4]]
parent_video_id: [["LAzKGkTIKpg","LAzKGkTIKpg","LAzKGkTIKpg","LAzKGkTIKpg","LAzKGkTIKpg"]]
parent_video_path: [["/home/zalasyu/Documents/projects/multimodal_chatbot/data/raw/videos/YouTube/LITERATURE_-_Voltaire_LAzKGkTIKpg.mp4","/home/zalasyu/Documents/projects/multimodal_chatbot/data/raw/videos/YouTube/LITERATURE_-_Voltaire_LAzKGkTIKpg.mp4","/home/zalasyu/Docu

In [43]:
df = video_segments_table.to_pandas()

In [44]:
df["embeddings_from_text"][4]

array([-1.89677011e-02,  3.87634188e-02, -4.88557331e-02,  2.08378900e-02,
       -1.60907805e-02,  1.19528512e-03, -8.22642632e-03, -6.14895485e-02,
        5.02643846e-02,  3.80183421e-02,  3.60288098e-02,  2.14225259e-02,
        4.70582722e-03,  3.26005295e-02, -3.40997078e-03, -1.39579009e-02,
       -9.14369598e-02, -6.08020276e-03, -2.11765841e-02,  2.89206114e-02,
        2.54447194e-04,  5.65460958e-02, -4.17123847e-02, -2.93858275e-02,
       -1.33415852e-02, -4.81536938e-03, -2.93757487e-02,  5.97635005e-03,
        2.46532690e-02, -3.68576264e-03,  1.39504690e-02, -9.78946406e-03,
       -3.59654278e-02,  2.28330027e-02,  5.99482330e-03,  1.69934593e-02,
        2.62959730e-02, -5.44480272e-02,  2.23583318e-02,  1.30270664e-02,
       -3.93417738e-02, -6.14391044e-02,  1.72194000e-02, -3.23664993e-02,
       -8.23974703e-03, -3.98813859e-02, -5.56484796e-03,  1.03183435e-02,
       -4.58829850e-02, -2.30268482e-02,  1.77339986e-02, -1.62703041e-02,
       -4.82203235e-04, -

## Query Testing

In [45]:
query = "Who did Voltaire try to save from execution?"
query_embedding = embedder.compute_query_embeddings(query)

In [46]:
len(query_embedding[0])

512

In [47]:
# Search for relevant video segments
results = (
    video_segments_table.search(query_embedding[0], vector_column_name="embeddings_from_text")
    .metric("cosine")
    .limit(3)
    .to_list()
)

In [48]:
for i, result in enumerate(results):
    print(f"Result {i}:")
    print(result["enriched_transcript"])
    print(result["_distance"])

Result 0:
Voltaire, who'd learned an important lesson about how change could be brought about through the pressure of public opinion. Opinion rules the world, he wrote in 1764, but in the long run, it's the philosophers who shape this opinion. Voltaire said of himself that he wrote to act, and he wanted his writings to change the way people thought and behaved. In leading his crusades against fanaticism, he even invented a campaign slogan, écraser la femme, which translates roughly as crush the despicable. La femme stands here for everything that Voltaire hated, everything that he had spent his life fighting, superstition, intolerance, irrational behaviour of every kind. We should never forget that Voltaire was also a brilliant writer, one of the greatest stylists the French language has ever known. The power of his ideas had a lot to do with the power of his expression. Many writers made fun of miracles, no one did so as hilariously as Voltaire. Always Voltaire had an ear for the tell